In [ ]:
!pip install kaggle
import kaggle

In [ ]:
#download dataset using kaggle api
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

In [ ]:
!pip install chardet

import chardet

# Read a small chunk of the file in binary mode
with open("orders.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # Reads first 100KB
    print(result)

In [ ]:
#read data from the file and handle null values
import pandas as pd
df = pd.read_csv("orders.csv", na_values = ['Not Available','unknown'])
df.head()

In [ ]:
df['Ship Mode'].unique()

In [21]:
#rename column names, make them lower case and replace space with underscore
df.columns
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')

In [ ]:
df.columns
df.head()

In [ ]:
#derive new columns discount, sale price and profit
df['discount'] = df['list_price']* df['discount_percent']* .01
df['sale_price'] = df['list_price'] - df['discount']
df['profit'] = df['sale_price'] - df['cost_price']
df

In [ ]:
#convert order date from object to datetime
df['order_date'] = pd.to_datetime(df['order_date'],format='%Y-%m-%d')
df.dtypes

In [35]:
#drop cost_price, list_price and discount_percent columns
df.drop(columns = ['list_price', 'cost_price', 'discount_percent'],inplace = True)

In [ ]:
df

In [ ]:
!pip install python-dotenv sqlalchemy psycopg2


In [ ]:
from sqlalchemy import create_engine
# from sqlalchemy.pool import NullPool
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Fetch variables
USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")

missing = [var for var in [USER, PASSWORD, HOST, PORT, DBNAME] if var is None]
if missing:
    raise ValueError(f"Missing environment variables: {missing}")


# Construct the SQLAlchemy connection string
# DATABASE_URL = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}?sslmode=require"

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}', future=True )

#If using Transaction Pooler or Session Pooler, we want to ensure we disable SQLAlchemy client side pooling -
# https://docs.sqlalchemy.org/en/20/core/pooling.html#switching-pool-implementations
# engine = create_engine(DATABASE_URL, poolclass=NullPool)

# Test the connection
try:
    with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print(f"Failed to connect: {e}")


In [ ]:
df.to_sql('orders', con=engine, if_exists='replace', index=False)